In [1]:
import tabula
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#
# Refactoring issues & ideas...
# ---------For short data, there are a total of 9 pages, 10 pages causes exception; 
# end of PDF short maximum/mormal length of lines read; anaylyze what the last sheet looks like
# too many pages? use "try:" to intercept the exception explosion

# QCMA large entries = 1000 properties or 32 PDF pages yields 971 data rows
# QCMA small entries = ~300 properties or  9 PDF pages yields 249 data rows

In [4]:
# Tabula reading performance
# Short =   9 total pages: 14 sec
# Long = 32 total pages: 47 sec


# Set up input and output file names...

# pdf_file_name = "pdfs\\raw_listing_data_approx_1000.pdf" # long data
# pkl_file_name = "DF_Long_Listings.pkl"

pdf_file_name = "pdfs\\raw_listing_data_approx_249.pdf" # short data
pkl_file_name = "DF_Short_Listings.pkl"
pdf_page_dfs = {}                      

for page in range(1,100):                                   
    pdf_page_dfs[page] = tabula.read_pdf(pdf_file_name, pages=page)
    if len(pdf_page_dfs[page].index) < 30: # checkign the number of lines in the PDF conversion\n",
        break

print(f'A total of {page} pages were read from the PDF file')

A total of 9 pages were read from the PDF file


In [5]:
cppdf = {} # cleaned PDF pages DF dictoinary
# tmpdf = {}
dummy = ""
splitstr = []
extrastr = []
tempostr = []
totalrowssaved2pages = 0

# totalistings = 0 use if last prage processsed
for page in range(1,len(pdf_page_dfs)): # ignore the last page for now
# for page in range(1,3):
    # the 3 categoroies of pages have different parsing requirements...
    print(f'Processing page {page}...')
    if page == 1:  
        # Prelim Clean up the original source DF...
        tmpdf = pdf_page_dfs[page].drop([0,1,2,3,4]).reset_index()
    else:
        tmpdf = pdf_page_dfs[page].drop([0,1,2,3]).reset_index()

    tmpdf.drop('index', axis=1, inplace=True)

    tmpdf.fillna('0', inplace = True)
#    DataFrame.fillna(self, value=None, method=None, axis=None, inplace=False, limit=None, downcast=None, **kwargs)[source]
# tmpdf.head()
    # create the DF for each page
    cppdf[page] = pd.DataFrame()
#   cppdf[page] = pd.DataFrame(columns=['MLS','Address', 'City','Zip','Beds','#Bath','#HlfBath','Gar','CvPk','TCP',\
#   'YB','Pool','SF','$/SF','Acres','List $','Sold $','Sold Date','SP%LP','CDOM'])
    if page == 1: # first page is different from the rest
        for row in range(0,len(tmpdf)):
            for col in range(0,6): #loop thjrough each field in the line 
#            for col in range(0,1): #loop thjrough each field in the line 
                splitstr = tmpdf.iloc[row,col].split(" ")
                if col == 0 and len(splitstr) < 4: # Check for bad row data: due to overflow of MLS & address line
                    break # break from column loop
                elif col == 0: # parse the first part of the PDF line AKA F_0
                    cppdf[page].at[row,'MLS'] = splitstr[0] # save the MLS# string
                    del splitstr[0] # remove MLS # so I can easily create the address string
                    cppdf[page].at[row,'Address'] = " ".join(splitstr) # create address string and load address
                elif col == 1:
                    cppdf[page].at[row,'Beds'] = splitstr[-1] # save the Beds # string
                    del splitstr[-1]                  # remove Beds # so I can easily create the City string
                    cppdf[page].at[row,'City'] = " ".join(splitstr) # Some cities compposed of multiple words ie. compund names
                    cppdf[page].at[row,'Zip'] = "0" # make a placeholder for Zip
                elif col == 2:
                    extrastr = splitstr[0].split("/")
                    cppdf[page].at[row,'#Bath'] = extrastr[0]
                    cppdf[page].at[row,'#HlfBath'] = extrastr[1] # create address string and load address
                elif col == 3:
                    cppdf[page].at[row,'YB'] = splitstr[1]
                    cppdf[page].at[row,'Pool'] = splitstr[2]
        
                    extrastr = splitstr[3].split(",") # extra slipt to handle thousands commas
                    cppdf[page].at[row,'SF'] = "".join(extrastr)
                    
                    
                    tempostr = splitstr[4].split("$") # from $/SF strip ""$"
                    del tempostr[0]                    
                    cppdf[page].at[row,'$/SF'] = tempostr[0]
                    
                    cppdf[page].at[row,'Acres'] = splitstr[5]
                    # extra split required to handle parking field with P/P/P
                    extrastr = splitstr[0].split("/")
                    cppdf[page].at[row,'Gar'] = extrastr[0]
                    cppdf[page].at[row,'CvPk'] = extrastr[1]
                    cppdf[page].at[row,'TCP'] = extrastr[2]
                elif col == 4:
                    cppdf[page].at[row,'List $'] = splitstr[0]
                else: 
                    
                    tempostr = splitstr[0].split("$") # from Sold $ strip $ and comma's
                    del tempostr[0]
                    extrastr = tempostr[0].split(",")
                    cppdf[page].at[row,'Sold $'] = "".join(extrastr)
                    
                    cppdf[page].at[row,'Sold Date'] = splitstr[1]
                    cppdf[page].at[row,'SP%LP'] = splitstr[2]
                    cppdf[page].at[row,'CDOM'] = splitstr[3]
            if col == 0 and len(splitstr) < 4: # Check for bad row data: due to overflow of MLS & address line
                continue # ignore this row and continue row loop
            totalrowssaved2pages += 1

    else: # after first page all the same format
        for row in range(0,len(tmpdf)):
            for col in range(0,10): #loop thjrough each field in the line 

                splitstr = tmpdf.iloc[row,col].split(" ")
    
                if col == 0 and len(splitstr) < 4: # Check for bad row data: due to overflow of MLS & address line
                    break # break from column loop
                
                elif col == 0: # parse the first part of the PDF line AKA F_0
                    cppdf[page].at[row,'MLS'] = splitstr[0] # save the MLS# string
                    del splitstr[0] # remove MLS # so I can easily create the address string
                    cppdf[page].at[row,'Address'] = " ".join(splitstr) # create address string and load address
                elif col == 1:
                    cppdf[page].at[row,'City'] = " ".join(splitstr) # Some cities are compound names
                    cppdf[page].at[row,'Zip'] = "0" # make a placeholder for Zip
                elif col == 2:
                    cppdf[page].at[row,'Beds'] = splitstr[0] # save the Beds # string
                elif col == 3:
                    extrastr = splitstr[0].split("/")
                    cppdf[page].at[row,'#Bath'] = extrastr[0] 
                    cppdf[page].at[row,'#HlfBath'] = extrastr[1] # create address string and load address
                elif col == 4:
                    # extra split required to handle parking field with P/P/P
                    extrastr = splitstr[0].split("/")
                    cppdf[page].at[row,'Gar'] = extrastr[0]
                    cppdf[page].at[row,'CvPk'] = extrastr[1]
                    cppdf[page].at[row,'TCP'] = extrastr[2]

                    cppdf[page].at[row,'YB'] = splitstr[1]
                    cppdf[page].at[row,'Pool'] = splitstr[2]

                elif col == 5:
                    extrastr = splitstr[0].split(",") # extra slipt to handle thousands commas
                    cppdf[page].at[row,'SF'] = "".join(extrastr)

                elif col == 6:
                    
                    tempostr = splitstr[0].split("$") # from $/SF strip ""$"
                    del tempostr[0]                    
                    cppdf[page].at[row,'$/SF'] = tempostr[0]

                elif col == 7:
                    cppdf[page].at[row,'Acres'] = splitstr[0]

                elif col == 8:
                    cppdf[page].at[row,'List $'] = splitstr[0]

                else: 
                    
                    tempostr = splitstr[0].split("$") # from Sold $ strip $ and comma's
                    del tempostr[0]
                    extrastr = tempostr[0].split(",")
                    cppdf[page].at[row,'Sold $'] = "".join(extrastr)
                    
                    cppdf[page].at[row,'Sold Date'] = splitstr[1]
                    cppdf[page].at[row,'SP%LP'] = splitstr[2]
                    cppdf[page].at[row,'CDOM'] = splitstr[3]
            if col == 0 and len(splitstr) < 4: # Check for bad row data: due to overflow of MLS & address line
                continue # ignore this row and continue row loop
            totalrowssaved2pages += 1
print(f'Total rows saved to individual PD pages: {totalrowssaved2pages}')

Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Total rows saved to individual PD pages: 249


In [6]:
# What I want it to look like
# apdf = pd.DataFrame(columns=\
# ['MLS','Address', 'City','Zip','Beds','#Bath','#HlfBath','Gar','CvPk','TCP','YB',\
#  'Pool','SF','$/SF','Acres','List $','Sold $','Sold Date','SP%LP','CDOM'])
# combine all the csvs into one..
apdf = pd.concat(cppdf, sort=False, ignore_index = True)
len(apdf)

249

In [7]:
# After the concat, the order of the columns needs to be restored
apdf = apdf[['MLS','Address', 'City','Zip','Beds','#Bath','#HlfBath','Gar','CvPk','TCP',\
             'YB','Pool','SF','$/SF','Acres','List $','Sold $','Sold Date','SP%LP','CDOM']]
apdf

,MLS,Address,City,Zip,Beds,#Bath,#HlfBath,Gar,CvPk,TCP,YB,Pool,SF,$/SF,Acres,List $,Sold $,Sold Date,SP%LP,CDOM
0,14101282,820 W Collins Street,Denton,0,2,1,1,0,0,0,1984,Yes,930,161.18,5.909,"$149,900",149900,07/05/2019,100.00,7
1,14092808,101 W Willow Street,Sanger,0,2,1,0,0,0,0,1997,No,1008,148.81,0.116,"$150,000",150000,07/10/2019,100.00,12
2,14081857,513 N Snyder Avenue,Justin,0,3,2,0,0,0,0,2008,No,1348,116.47,0.165,"$159,900",157000,07/12/2019,98.19,19
3,14015075,601 Mary Street,Aubrey,0,3,1,0,0,0,0,1977,No,858,186.48,0.170,"$169,900",160000,07/11/2019,94.17,111
4,14085452,4152 Caldwell Avenue,The Colony,0,3,2,0,2,0,2,1986,No,1492,110.59,0.127,"$179,900",165000,07/09/2019,91.72,42
5,14092244,214 Southside Drive,Sanger,0,3,1,0,0,0,0,1992,No,1268,130.13,0.190,"$169,000",165000,07/10/2019,97.63,19
6,14105581,303 W Pecan Street,Aubrey,0,4,2,0,0,0,0,1996,No,1405,122.42,0.172,"$169,900",172000,07/08/2019,101.24,3
7,14085089,724 E Roewe Street,Pilot Point,0,3,2,0,2,0,2,1978,No,1177,148.68,0.272,"$169,900",175000,07/10/2019,103.00,8
8,14112097,2742 Harbinger Lane,Dallas,0,2,1,0,1,0,1,1984,No,819,219.78,0.061,"$180,000",180000,07/11/2019,100.00,3
9,14086107,527 N 1st Street,Krum,0,3,2,0,2,0,2,1999,No,1235,147.37,0.227,"$184,900",182000,07/08/2019,98.43,28


In [8]:
# convert from strings to numbers: pool
# Columns we wont use: $/SF, List $, 

In [9]:
# Ignore these, until we might need them
# MLS          object
# Address      object
# City         object
# Zip          object

apdf['Beds'] = apdf['Beds'].astype(int)
apdf['#Bath'] = apdf['#Bath'].astype(int)
apdf['#HlfBath'] = apdf['#HlfBath'].astype(int)
apdf['Gar'] = apdf['Gar'].astype(int)

# CvPk         object <-- this gave trouble
apdf.drop(columns='CvPk', axis=0, inplace = True) # Production fix? : fix nulls then keep column
apdf['TCP'] = apdf['TCP'].astype(int)
apdf['YB'] = apdf['YB'].astype(int)

In [10]:
# apdf.pool.replace(('yes', 'no'), (1, 0), inplace=True) does not work
# poolmap = {'Yes': True, 'No': False} --> stays "object"
poolmap = {'Yes': 1, 'No': 0} # converts to float
apdf['Pool'] = apdf['Pool'].map(poolmap) # better but not perfect: int64 == overkill; why not BOOL ??

In [11]:
apdf['SF'] = apdf['SF'].astype(int)
apdf['$/SF'] = apdf['$/SF'].astype(float)
apdf['Acres'] = apdf['Acres'].astype(float)
apdf['Sold $'] = apdf['Sold $'].astype(int)
# Sold Date    object
# SP%LP        object
# CDOM         object

In [12]:
# By now everything we expoect to use with the ML model should be numeric
apdf.dtypes

MLS           object
Address       object
City          object
Zip           object
Beds           int32
#Bath          int32
#HlfBath       int32
Gar            int32
TCP            int32
YB             int32
Pool           int64
SF             int32
$/SF         float64
Acres        float64
List $        object
Sold $         int32
Sold Date     object
SP%LP         object
CDOM          object
dtype: object

In [13]:
apdf['Sold $'].describe()

count    2.490000e+02
mean     3.666548e+05
std      1.724461e+05
min      1.499000e+05
25%      2.550000e+05
50%      3.145000e+05
75%      4.228890e+05
max      1.100000e+06
Name: Sold $, dtype: float64

In [14]:
# apdf['Pool'] = apdf['Pool'].astype(bool) # this made all Pools true ?!?!?

In [15]:
apdf.dtypes

MLS           object
Address       object
City          object
Zip           object
Beds           int32
#Bath          int32
#HlfBath       int32
Gar            int32
TCP            int32
YB             int32
Pool           int64
SF             int32
$/SF         float64
Acres        float64
List $        object
Sold $         int32
Sold Date     object
SP%LP         object
CDOM          object
dtype: object

In [16]:
apdf

,MLS,Address,City,Zip,Beds,#Bath,#HlfBath,Gar,TCP,YB,Pool,SF,$/SF,Acres,List $,Sold $,Sold Date,SP%LP,CDOM
0,14101282,820 W Collins Street,Denton,0,2,1,1,0,0,1984,1,930,161.18,5.909,"$149,900",149900,07/05/2019,100.00,7
1,14092808,101 W Willow Street,Sanger,0,2,1,0,0,0,1997,0,1008,148.81,0.116,"$150,000",150000,07/10/2019,100.00,12
2,14081857,513 N Snyder Avenue,Justin,0,3,2,0,0,0,2008,0,1348,116.47,0.165,"$159,900",157000,07/12/2019,98.19,19
3,14015075,601 Mary Street,Aubrey,0,3,1,0,0,0,1977,0,858,186.48,0.170,"$169,900",160000,07/11/2019,94.17,111
4,14085452,4152 Caldwell Avenue,The Colony,0,3,2,0,2,2,1986,0,1492,110.59,0.127,"$179,900",165000,07/09/2019,91.72,42
5,14092244,214 Southside Drive,Sanger,0,3,1,0,0,0,1992,0,1268,130.13,0.190,"$169,000",165000,07/10/2019,97.63,19
6,14105581,303 W Pecan Street,Aubrey,0,4,2,0,0,0,1996,0,1405,122.42,0.172,"$169,900",172000,07/08/2019,101.24,3
7,14085089,724 E Roewe Street,Pilot Point,0,3,2,0,2,2,1978,0,1177,148.68,0.272,"$169,900",175000,07/10/2019,103.00,8
8,14112097,2742 Harbinger Lane,Dallas,0,2,1,0,1,1,1984,0,819,219.78,0.061,"$180,000",180000,07/11/2019,100.00,3
9,14086107,527 N 1st Street,Krum,0,3,2,0,2,2,1999,0,1235,147.37,0.227,"$184,900",182000,07/08/2019,98.43,28


In [17]:
# If adding GPS coords, put this here...
# To obtain GPS coordinates of an address, begin here to get Google API
# https://cloud.google.com/maps-platform/?apis=places
# edf.insert(4, 'GPS_Lat', 0, allow_duplicates=False)
# edf.insert(5, 'GPS_Lon', 0, allow_duplicates=False)
# edf.head()

In [18]:
apdf['Sold $'].describe()

count    2.490000e+02
mean     3.666548e+05
std      1.724461e+05
min      1.499000e+05
25%      2.550000e+05
50%      3.145000e+05
75%      4.228890e+05
max      1.100000e+06
Name: Sold $, dtype: float64

In [19]:
apdf['Pool'].describe()

count    249.000000
mean       0.164659
std        0.371619
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: Pool, dtype: float64

In [20]:
apdf['City'].describe()

count        249
unique        33
top       Denton
freq          32
Name: City, dtype: object

In [21]:
print(len(apdf['City'].unique()))
apdf['City'].unique()

33


array(['Denton', 'Sanger', 'Justin', 'Aubrey', 'The Colony',
       'Pilot Point', 'Dallas', 'Krum', 'Corinth', 'Providence Village',
       'Carrollton', 'Little Elm', 'Lewisville', 'Cross Roads',
       'Flower Mound', 'Frisco', 'Savannah', 'Ponder', 'Oak Point',
       'Lantana', 'Fort Worth', 'Trophy Club', 'Prosper', 'Argyle',
       'Lakewood Village', 'Hickory Creek', 'Highland Village',
       'Northlake', 'Coppell', 'Double Oak', 'Roanoke', 'Copper Canyon',
       'Southlake'], dtype=object)

In [22]:
apdf['Address'].describe()

count                         249
unique                        249
top       1448 Hollow Ridge Drive
freq                            1
Name: Address, dtype: object

In [23]:
apdf['Acres'].describe()


count    249.000000
mean       0.478008
std        1.998766
min        0.000000
25%        0.140000
50%        0.180000
75%        0.233000
max       28.000000
Name: Acres, dtype: float64

In [24]:
apdf.to_pickle(pkl_file_name) # Save data for use by rempan & rempest.ibynb